# Analyse des Précipitations Annuelles Cumulées : Une Exploration des Données Météo de Météo France


Les données météorologiques constituent une ressource précieuse pour comprendre les tendances climatiques et les phénomènes météorologiques qui affectent notre planète. Dans ce notebook, nous allons explorer le fichier de données météo fourni par Météo France, qui contient des informations quotidiennes sur les précipitations, les températures et d'autres variables météorologiques pour divers postes en France, couvrant une période allant de 1950 à 2023. Le fichier source est disponible à l'adresse suivante : https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz.

L'objectif de cette analyse est de calculer la somme cumulative des précipitations journalières moyennes sur toutes les stations à la manière d'une intégrale depuis le début de chaque année et de ne garder que la valeur de fin de chaque année. Cela nous permettra de comprendre l'évolution des précipitations annuelles cumulées sur la période étudiée et d'identifier les tendances et les variations significatives.


## Méthodologie

Pour atteindre cet objectif, nous allons utiliser la bibliothèque DuckDB pour traiter les données et effectuer les calculs nécessaires. Nous allons d'abord charger les données dans DuckDB, puis nous allons utiliser une requête SQL pour calculer la moyenne des précipitations journalières par jour, suivie d'un calcul de la somme cumulative de ces moyennes par année. Nous allons ensuite utiliser Python et les bibliothèques Matplotlib et Pandas pour visualiser les résultats et mettre en évidence les tendances et les variations dans les précipitations annuelles cumulées.


## Résultats

Les résultats de l'analyse seront présentés sous forme de graphique, montrant l'évolution des précipitations annuelles cumulées sur la période étudiée, ainsi que la courbe lissée sur 20 ans pour mettre en évidence les tendances à long terme.


## Visualisation

La visualisation des résultats sera effectuée à l'aide de Matplotlib, avec une mise en forme soignée pour faciliter la lecture et l'interprétation des données.


Ce notebook Jupyter est conçu pour être une ressource informative et pédagogique, accessible à un public intéressé par l'analyse de données et les sciences de l'environnement. Nous espérons qu'il inspirera les lecteurs à explorer davantage les données météorologiques et à approfondir leur compréhension des phénomènes climatiques.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH daily_avg_rr AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS annee,
    CAST(SUBSTRING("AAAAMMJJ", 5, 2) AS INTEGER) AS mois,
    CAST(SUBSTRING("AAAAMMJJ", 7, 2) AS INTEGER) AS jour,
    AVG(CAST("RR" AS DOUBLE)) AS avg_rr
  FROM loaded_dataset
  GROUP BY "AAAAMMJJ"
),
cumulative_sum AS (
  SELECT 
    annee,
    jour,
    mois,
    SUM(avg_rr) OVER (PARTITION BY annee ORDER BY mois, jour) AS cum_sum_rr
  FROM daily_avg_rr
)
SELECT 
  annee, 
  cum_sum_rr AS integral_precipitation
FROM cumulative_sum
WHERE mois = 12 AND jour = 31
ORDER BY annee """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 74 lignes


,annee,integral_precipitation
0,1950,1354.693791
1,1951,1537.841796
2,1952,1583.263803
3,1953,915.175000
4,1954,1498.891898


## 📈 Visualisation

La bibliothèque principale utilisée est Matplotlib, qui est une des bibliothèques de visualisation de données les plus populaires en Python. Elle est adaptée pour créer des graphiques de haute qualité avec une grande flexibilité dans la personnalisation, ce qui est idéal pour représenter des données temporelles comme les précipitations annuelles cumulées. Matplotlib permet une visualisation claire et détaillée des données, avec des options de mise en forme avancées.

In [4]:
import pandas as pd
import duckdb as ddb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Source Sans Pro'
mpl.rcParams['font.size'] = 10

# Mise en ordre
df = df.sort_values('annee').reset_index(drop=True)

# Calcul du lissage 20 ans
window = min(20, len(df) - (len(df) % 2 == 0))
polyorder = min(3, window - 1)
df['precip_lisse'] = savgol_filter(df['integral_precipitation'], window_length=window, polyorder=polyorder)

# Préparation figure
fig, ax = plt.subplots(figsize=(12, 6))
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Courbes
ax.plot(df['annee'], df['integral_precipitation'], 
        color='#ff6666', linewidth=1.2, label='Données brutes', alpha=0.8)
ax.plot(df['annee'], df['precip_lisse'], 
        color='#1f77b4', linewidth=2.5, label='Courbe lissée 20 ans')

# Points min/max
min_idx = df['integral_precipitation'].idxmin()
max_idx = df['integral_precipitation'].idxmax()
for idx, color in [(min_idx, 'red'), (max_idx, 'red')]:
    ax.scatter(df.at[idx, 'annee'], df.at[idx, 'integral_precipitation'], 
               color=color, s=60, zorder=5, edgecolors='white', linewidth=1.5)
    ax.annotate(f"{df.at[idx, 'integral_precipitation']:.0f}", 
                xy=(df.at[idx, 'annee'], df.at[idx, 'integral_precipitation']),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor=color, alpha=0.7),
                fontweight='bold', clip_on=True)

# Mise en forme
ax.set_title('Précipitations annuelles cumulées lissées sur 20 ans', pad=20, fontsize=14, fontweight='bold')
ax.set_xlabel('Années (1950-2023)', labelpad=10)
ax.set_ylabel('Précipitations cumulées (mm)', labelpad=10)
ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
ax.set_xlim(df['annee'].min() - 1, df['annee'].max() + 1)
legend = ax.legend(loc='upper left', frameon=True, fancybox=True, shadow=True)
legend.get_frame().set_facecolor('white')
legend.get_frame().set_edgecolor('lightgray')
plt.tight_layout()

dataviz = fig
dataviz

ModuleNotFoundError: No module named 'scipy'

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250820_222740.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250820_222740.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


AVERTISSEMENT: Aucune variable 'dataviz' trouvée.
